<h1>Image crawler</h1>

<div>In this notebook, we are going to find pictures of Anne Shirley's quotes by crawling the web, extract the text of the pictures and translate them into Persian, then paste the translated text on the new pictures. We do this process by searching the web using image crawlers. Many tools can be found for this task provided by various developers, whatever we use in this notebook also mention its source. We do this in the following steps.</div>

<ul style='list-style-type:square'>
    <li>Crawling Anne Shirley's quotes</li>
    <li>Extracting text from images</li>
    <li>Spell checking</li>
    <li>Cleaning the texts</li>
    <li>Checking the validation of the text</li>
    <li>Translating</li>
    <li>Crawling new another images</li>
    <li>Inserting the texts in the context of new images</li>
    <li>Display the results</li>
</ul>

<b>Introduction</b><br/>
<style>  
  div {padding-right:25px; padding-left: 25px;}  
</style>
<div>A Web crawler, sometimes called a <b>spider</b> or <b>spiderbot</b> and often shortened to <b>crawler</b>, is an Internet bot that systematically browses the World Wide Web and that is typically operated by search engines for the purpose of Web indexing (web spidering).<a href="https://en.wikipedia.org/wiki/Web_crawler#cite_note-1">[1]</a></div><br/>
<div>The image crawler application is used to collect a multitude of images from websites. The images can be viewed as thumbnails or saved to a given folder for enhanced processing.<a href="https://sourceforge.net/projects/imagecrawler/">[2]</a><br/>
</div>

<b>Contents</b><br/>
<ul>
  <li>Installations</li>
      <ul>
          <li>icrawler</li>
          <li>pytesseract</li>
          <li>pyspellchecker</li>
          <li>fuzzywuzzy</li>
          <li>deep-translator</li>
          <li> if necessary, install arabic_reshaper and python-bidi</li>
      </ul>
  <li>Imporl libraries</li>
  <li>Implementation helper functions</li>
  <li>Crawl</li>
</ul>

<h1><b>Installations</b></h1>

<b>Install icrawler</b><br/>
In this project, we use the icrawler library, if you haven't installed it yet, you should install it(Python 3.5+ recommended).<br/>
With this package, you can write a multiple thread crawler easily by focusing on the contents you want to crawl, keeping away from troublesome problems like exception handling, thread scheduling and communication.<a href='https://pypi.org/project/icrawler/'>[3]</a>

It also provides built-in crawlers for popular image sites like Flickr and search engines such as Google, Bing and Baidu.<a href='https://pypi.org/project/icrawler/'>[4]</a>

In [ ]:
# if you haven't installed it yet, you should install it with the following command.
# Run this line just first time
%pip install icrawler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<h1>Install pytesseract</h1><br/>
For enabling our python program to have character recognition capabilities, we would be making use of pytesseract OCR library. The library could be installed onto our python environment by executing the command "%pip install pytesseract" in the command interpreter of the OS.<a href="https://www.geeksforgeeks.org/how-to-extract-text-from-images-with-python/">[5]</a>

The library (if used on Windows OS) requires the tesseract.exe binary to be also present for proper installation of the library. During the installation of the aforementioned executable, we would be prompted to specify a path for it. This path needs to be remembered as it would be utilized later on in the code.<a href="https://www.geeksforgeeks.org/how-to-extract-text-from-images-with-python/">[6]</a>

Normally Tesseract has been developed on Debian GNU Linux, but there is also the need for a Windows version. That's why has been built a Tesseract installer for Windows.<a href="https://github.com/UB-Mannheim/tesseract/wiki#tesseract-installer-for-windows">[7]</a>

Therefore, to use in Windows, download its installer file from <a href="https://github.com/UB-Mannheim/tesseract/wiki#tesseract-installer-for-windows">here</a>.

<b>WARNING:</b> After installation, To use this ocr correctly, the pytesseract.tesseract_cmd variable must be assigned the installation path (this path is used by the library to find the executable and use it for extraction).

In [ ]:
!sudo apt install tesseract-ocr
%pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 2s (2,949 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

<b>pyspellchecker</b>

In [ ]:
%pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.5 MB/s eta 0:00:00


<b>fuzzywuzzy</b>

In [ ]:
%pip install fuzzywuzzy
%pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.1 MB/s eta 0:00:00


<b>deep-translator</b>

In [ ]:
%pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<b>arabic_reshaper and python-didi</b>

In [ ]:
#!pip install arabic_reshaper
#!pip install python-bidi

<h1><b>Import libraries</b></h1>

In [ ]:
import datetime
import os,shutil
import re
#Options for spell cheking
# %pip uninstall PyEnchant
# %pip install textblo
# %pip install jamspell
# %pip install autocorrect
from spellchecker import SpellChecker
from fuzzywuzzy import fuzz
from icrawler.builtin import bing
from icrawler.builtin import google 
#Other options
# from icrawler.builtin import baidu
# from icrawler.builtin import flickr

from typing import Literal
#import arabic_reshaper
#from bidi.algorithm import get_display
from string import ascii_letters
import textwrap
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from deep_translator import GoogleTranslator
import pytesseract


<h1>Initailize</h1>

In [ ]:
workspace = ""
root_directory =""
quote_path = ""
output_path=""
wallpaper_keyword = "sunny"
anne_keyword="AnneShirleyQuotes"

max_number = 35
threshold = 75

pytesseract.pytesseract.tesseract_cmd =(r'/usr/bin/tesseract')
#Whe local: 'C:/Program Files/Tesseract-OCR/tesseract.exe' 

translator = GoogleTranslator(source='auto', target='fa')

spell = SpellChecker()

<h1><b>Implementation helper functions</b></h1>

Implementation of get_crawled_image_names

In [ ]:
# returns crawled images directory and list of images in
def get_crawled_image_names(parent_path=None):
    # get list of crawled images
    # Note: crawled imaged by icrawler is stored in a folder named as 'Images'
    # in root directory
    list_ = os.listdir(parent_path)
    # filter files in the 'Images' folder by valid extensions
    ext = ['jpg','jpeg','png','bmp']

    # Filter file names based on given extensions in list 2
    filtered = [x for x in list_ if any(y in x for y in ext)]
    
    # Returns crawled image names as list
    return filtered


Implementation of text cleaning

In [ ]:
extra_spaces_pattern = re.compile('\s+')

def clean_text(text=""):
    # Step 1: replace linebreak characters by white space
    text = text.replace('\n',' ')
    # Step 2: Replace extra white spaces by single white space
    text = re.sub(extra_spaces_pattern, ' ', text)
    #Step 3: Unnecessary characters Removal
    text = re.sub(r"[^a-zA-Z0-9',]", " ", text) 
    
    if text ==" ": text =""

    return text


Implementation of fuzzy search

In [ ]:
def fuzzy_match(str1="",book_path=None):
    best_file=""
    best_match = 0
    list_ = os.listdir(book_path)
    for f in list_:
        path_ = book_path +"/"+ str(f) 
        file = open(path_,"r")

        content = file.read()
        ratio = fuzz.token_set_ratio(str1, content)
        if best_match < ratio:
           best_match = ratio
           best_file = f

    return best_match, best_file
    

Implementation of spell_check

In [ ]:

def spell_check(text=""):
    words = text.split()
    final_text=""
    for word in words:
        misspelled = spell.unknown([word])
        if len(misspelled)==1:
           correct = spell.correction(word)
           if correct == None:
              final_text = final_text+ " "+ word
           else:
              final_text = final_text+" "+ spell.correction(word)
        else:
            # Ignore from first empty space
            if final_text =="":
               final_text = word
            else:
               final_text = final_text+ " "+ word
    
    return final_text


Implementaion toggle_safe_mode

In [ ]:
def toggle_safe_mode():

      msg ="Two modes are possible for crawling.\n\n"\
           "\033[1mAutomatic:\033[0m in this mode, the program automatically explores "\
           "the web and collects images up to \na maximum number specified by the user, "\
           "in this mode, \033[1munethical\033[0m images may also be discovered.\n\n"\
           "\033[1mSafe:\033[0m This is a controlled mode, in which 35 images that have "\
           "already been collected are processed.\n"

      print(msg)

      print("Proceed safely?(\033[1mY/N\033[0m)")

      mode = input()
      
      return mode


Implementation auto_crawl

In [23]:
#Valid toot path must be is 'workspace' or 'workspace' + 'safe_images/'
def auto_crawl(max_crawl = 20,quote_path=None,threshold=80, font = None):
  """
  Returns a dictiunary {
                        "undiscovered":string list of file names,
                        "ignored":{"match": float,"image_file":string},
                        "processed":list of {"source_file":string , "wallpaper":string , "generated":string}
                        }
  """
  # The path of the files from which no text is detected is saved here.
  undiscovered_files=[]
  # The path of the files whose discovered citation matches are less 
  # than the specified threshold is stored.
  ignored_img_list = []
  # The path of the files that were processed correctly is saved.
  processed_images = []

  # Get image names stored in anne shirley qoutes folder
  anne_img_names = get_crawled_image_names(quote_path+"/anne_quotes")
  
  # Get image names stored in wallpapers folder
  wallpaper_img_names = get_crawled_image_names(quote_path+"/wallpapers")
  
  image_count = min(len(anne_img_names), len(wallpaper_img_names))

  if image_count < max_crawl: max_crawl = image_count

  for i in range(max_crawl):

      anne_img_name      = quote_path + "/anne_quotes/" + anne_img_names[i]

      wallpaper_img_name = quote_path + "/wallpapers/" + wallpaper_img_names[i]

      generated_img_name = output_path +"/"+ str(i + 1)+".jpg"

      anne_img = Image.open(anne_img_name)
      # detect text from the image
      text = pytesseract.image_to_string(image = anne_img , lang='eng')

      # Cleaning text 
      text = clean_text(text = text)
      
      if text == "": 
        undiscovered_files.append(anne_img_name)
        continue

      # Spell Cheching
      final_text = spell_check(text=text)
      
      # Step 3: Call fuzzy_match to check quote validation in anne books
      obtained_match,book = fuzzy_match(str1 = final_text,book_path= anne_book_path)

      # If matching ratio is less than 90% we ignore this quote and go to next quote
      if obtained_match < threshold :

        ignored_img_list.append({"match": obtained_match,"image_file":anne_img_name})

        print()
        print("Ignored due to less than "+ str(threshold) +"% match.\n"+anne_img_name)
        print("\033[1mThe obtained match " + str(obtained_match)+"%\033[0m")
        print()
        continue # go to next quote 
      # Step 3: Translate the text to Persian
      translated = translator.translate(text=final_text)

      #   End of cleaning text 
      #_____________________________________________________________________________________________
      
      # Step 4: Generate output image
      
      # Step 4.1: Open image from wallpapers
      out_img = Image.open(wallpaper_img_name)
      #_____________________________________________________________________________________________
      # Text placement cumputation 
    
      # Calculate the average length of a single character of our font.
      # Note: this takes into account the specific font and font size.
      avg_char_width = sum(font.getsize(char)[0] for char in ascii_letters) / len(ascii_letters)
      # Translate this average length into a character count
      # to fill 95% of our image's total width
      max_char_count = int( (out_img.size[0] * .95) / avg_char_width )

      # End of Text placement computation
      #_____________________________________________________________________________________________

      # Create a wrapped text object using scaled character count
      scaled_wrapped_text = textwrap.fill(text=translated, width = max_char_count)

      #print("\033[1mFrom Image "+str(i+1)+": \033[0m",text)
      #print("\033[1mTranslated: \033[0m",scaled_wrapped_text)
      #print()#empty line 
      
      # Step 4.2: Call draw Method to add 2D graphics in an image
      draw = ImageDraw.Draw(out_img)

      # Step 4.3: insert text to the Draw object

      #  4.3.1: make shape of text
      # correct its shape
      #reshaped_text = arabic_reshaper.reshape(translated)    
      
      #  4.3.2: make direction of text
      # correct its direction
      #bidi_text = get_display(reshaped_text)                            
      
      # 4.3.3 Calculate center point of text placement
      width  = out_img.width
      height = out_img.height
      img_center = (width/2,height/2)
      
      #  4.3.4: Draw text in image
      draw.text(xy = img_center,
                text = scaled_wrapped_text,
                fill="Black",
                stroke_fill="Gold",
                stroke_width=10,
                font=font,
                direction="rtl",
                language='fa',
                anchor ='mm',
                Literal='center',
                align='right')
      
      # Step 5: Save output in Generated-Image path
      out_img.save(generated_img_name)
      
      anne_img_name = anne_img_name.replace(work_directory,"")
      wallpaper_img_name = wallpaper_img_name.replace(work_directory,"")
      generated_img_name = generated_img_name.replace(work_directory,"")

      processed_images.append({"source_file":anne_img_name,
                                                     "wallpaper":wallpaper_img_name ,
                                                     "generated":generated_img_name})

      print("Generated image #",generated_img_name)
  
  return {"undiscovered":undiscovered_files,"ignored":ignored_img_list,"processed":processed_images}


Implementation generate_html_table

In [31]:
def generate_html_table(processed_data=None,directory_name=None):

    print("Generateing HTML table started ...")

    if processed_data == None:
         print("Stoped: No processed data found!")
         return
    
    count = len(processed_data)
    str_rows=""
    row_header = "<tr>\n"\
                           "\t<td align='center'><h1>Quotes</h1></td>\n"\
                           "\t<td align='center'><h1>Wallpapers</h1></td>\n"\
                           "\t<td align='center'><h1>Generated</h1></td>\n"\
                           "</tr>"

    for i in range(count):

          source = processed_data[i]["source_file"]
          # remove parent directory
          #source = source.replace(work_directory,'')

          wallpaper=processed_data[i]["wallpaper"]
          #wallpaper = wallpaper.replace(work_directory,"")

          generated=processed_data[i]["generated"]
          #generated = generated.replace(work_directory,'')
          row_str ="<tr>\n"\
                         "\t<td><img src='."+source+"' width='300' height='200'/></td>\n"\
                         "\t<td><img src='."+wallpaper+"' width='300' height='200'/></td>\n"\
                         "\t<td><img src='."+generated+"' width='300' height='200'/></td>\n"\
                         "</tr>"
          
          str_rows = str_rows + row_str
    
    print("Collected " +str(count)+" images ...")

    header_text = "<h1>Image crawler</h1><br/>\n" \
                           "<b>Abdullah Mohammadi<br/>abdhmohammady@gmail.com</b><br/><br/>\n"\
                           "The result of this table is collected based on crawling the web with the term 'AnneShirleyQuotes'.<br/>\n" \
                           "After collecting about 100 images with the theme of 'Quotes of Anne Shirley', we extracted the<br/>\n"\
                           "text of the images and placed the Persian translation on a new image.<br/><br/>\n"\
                           "In the table below:<br/>\n"\
                           "The 'Quotes' column from the search results of 'AnneShirleyQuotes',<br/>\n"\
                           "The 'Wallpapers' column from the search results of 'sunny' and<br/>\n"\
                           "The 'Generated' column are processed results.<br/><br/>\n"

    table_str = "<table align='center' border='1'>\n" + row_header + str_rows + "\n</table>"

    html_text = header_text + table_str
    
    #hs = open(directory_name+"/RESULTS.html", 'w')
    #hs.write(html_text)
    #hs.close() 
    try:
      readme = open(directory_name+"/README.MD", 'w')
      readme.write(html_text)
      readme.close() 

      print("HTML table saved in:", directory_name+"/README.MD")
    except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")

      print()


Implementation of remove_folder_content

In [ ]:
def remove_folder_content(folder=None):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

<h1><b>Main</b></h1>

In [30]:
#Toggles Safe mode and Automatic Made
mode =  toggle_safe_mode()
# Automatic: in this mode, the program automatically explores the web and collects 
# images up to a maximum number specified in the 'Initalization' cell.
# in this mode,unethical images may also be discovered.
# Safe: This is a controlled mode, in which 35 images that have already been collected are processed.

n = datetime.datetime.now()

print ("Process started at %s:%s:%s" % (n.hour, n.minute, n.second))

#keywords = ["Y","y","N","n"]

print("Mode: ",mode)

print("Max number:", max_number)

print("Threshold:",threshold)

if mode not in ["Y","y","N","n"]:
      print("Terminate by wrong keywords!")
      exit()

workspace = os.getcwd()
# Calling os.getcwd() has the following results:
# In google colab it returns '/content', This is the root directory.
# In GitHub codespace returns '/workspaces/codespaces-jupyter' 
# but this is the work directory.

# In GitHub codespace, our work directory is '/workspaces/codespaces-jupyter'
work_directory = workspace +""

# ENABLE THIS LINE IN 'Google Colab' WORKSPACE
# DISABLE THIS LINE IN GitHub' CODESPACE
# In google colab, our work directory is '/content/drive/MyDrive/Colab Notebooks/Text-Mining'
work_directory = workspace + "/drive/MyDrive/Colab Notebooks/Text-Mining"

# When workspace and work directory was specified ...
font_path = work_directory+"/fonts"
anne_book_path = work_directory+"/anne_books"
# Directory to generate wallpapers from crawled images
output_path = work_directory+"/generated_images"

if not os.path.exists(output_path): os.makedirs(output_path)
else: remove_folder_content(output_path)

# Safe crawling is requested
if mode in ["Y","y"]:
   print("Safe mode selected.")
   # Defualt 'anne_quote' and 'wallpapers' folders located here.
   quote_path = work_directory + "/safe_images"
# Automatic crawling is requested, In automatic mode:
# In the first step, the images of Anne Shirley are searched
# In the second name step, the folder of the found images is renamed to the folder of 'anne_quotes.
# In the third step, wallpaper images are searched
# In the fourth step, the name of the folder of the found images is changed to that of 'wallpapers'.
elif mode in ["N","n"]:
   print("Automatic mode selected.")
   quote_path = workspace
   # When 'anne qoutes' crawled,by defualt,crawler creates 'images' folder in root directory
   # to manage root diregtory, we store it in 'workspace' varible.
   # Clear content of 'images' folder
   if os.path.exists(quote_path+"/images"):
      #remove_folder_content(workspace+"/images")
      # deletes the directory and all its contents.
      shutil.rmtree(quote_path +"/images")
   
   # Step 1: Crawling starts
   bing_crawler = bing.BingImageCrawler()
   bing_crawler.crawl(keyword = anne_keyword, offset = 0, max_num = max_number, min_size = None, max_size = None)
   #by default the crawler creates a folder as 'images' in root directory
   # and stores crawled images in, we rename this folder to 'anne_quotes'
   # becuase we want crawl wallpapers again.
    
   # Step 2: Rename crawled 'images' folder to 'anne_quotes'
   if os.path.exists(quote_path +"/anne_quotes"):
      #remove_folder_content(workspace +"/anne_quotes")
      shutil.rmtree(quote_path +"/anne_quotes")

   os.rename(quote_path+"/images",quote_path + "/anne_quotes")
   
   # Step 3: Wallpapers Crawling starts
   #google_crawler = google.GoogleImageCrawler()
   bing_crawler.crawl(keyword = wallpaper_keyword, offset = 0, max_num = max_number, min_size = None, max_size = None)
   #by default the crawler creates a folder as 'images' in root directory
   # and stores crawled images in, we rename this folder to 'wallpapers'
   
   # Step 4: Rename crawled 'images' folder to 'wallpapers'
   if os.path.exists(quote_path +"/wallpapers"):
      shutil.rmtree(quote_path +"/wallpapers")
   
   if os.path.exists(quote_path+"/images"):
      os.rename(quote_path+"/images",quote_path +"/wallpapers")

font_ = ImageFont.truetype(font = font_path +'/Vazirmatn-Regular.ttf',size= 70)

result = auto_crawl(max_crawl= max_number , quote_path = quote_path , threshold=threshold, font = font_)

processed = len(result["processed"])

ignored = len(result["ignored"])

undiscovered = len(result["undiscovered"])

print()
print("-------------------------------------------------------------------------------------")
print("\033[1mText matching threshold\033[0m:",threshold)
print("\033[1mProcessed images\033[0m:",processed)
print("\033[1mIgnored images\033[0m:",ignored)

for item in result["ignored"]:
    print(item["image_file"]+", obtained ratio : ",str(item["match"])+"%")

print("\033[1mUndiscovered images\033[0m : ",undiscovered)

for item in result["undiscovered"]:
       print(item)

generate_html_table(processed_data = result["processed"],directory_name = work_directory)

n = datetime.datetime.now()
print ("All tasks done at %s:%s:%s" % (n.hour, n.minute, n.second))




Two modes are possible for crawling.

Automatic: in this mode, the program automatically explores the web and collects images up to 
a maximum number specified by the user, in this mode, unethical images may also be discovered.

Safe: This is a controlled mode, in which 35 images that have already been collected are processed.

Proceed safely?(Y/N)
y
Process started at 16:46:9
Mode:  y
Max number: 2
Threshold: 75
Safe mode selected.
Generated image # /generated_images/1.jpg
Generated image # /generated_images/2.jpg

-------------------------------------------------------------------------------------
Text matching threshold: 75
Processed images: 2
Ignored images: 0
Undiscovered images :  0
Generateing HTML table started ...
<tr>
	<td><img src='./safe_images/anne_quotes/32.jpg' width='300' height='200'/></td>
	<td><img src='./safe_images/wallpapers/1.jpg' width='300' height='200'/></td>
	<td><img src='./generated_images/1.jpg' width='300' height='200'/></td>
</tr>
<tr>
	<td><img src='./